In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import stellargraph as sg
from stellargraph import StellarGraph
from stellargraph.data import EdgeSplitter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV

2023-03-11 03:51:51.519967: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-11 03:51:51.734185: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-11 03:51:51.739266: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-11 03:51:51.739301: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
def filter_Pikachu(df):
    df = df.assign(src_user = df['source user@domain'].str.split('@').map(lambda x: x[0]))
    df = df[(df['src_user'].str.contains('LOCAL')==False) & (df['src_user'].str.contains('SYSTEM')==False) & (df['src_user'].str.contains('ANONYMOUS')==False) & (df['src_user'].str.contains('ADMINISTRARTOR')==False)]
    df = df[df['src_user'].str[-1] != '$']
    return df

In [3]:
red_df = pd.read_csv('../../../lm-vol/redteam_complete.csv')
red_df.rename(columns = {'source':'source user@domain', 'source_computer': 'source computer', 'destination_computer': 'destination computer'}, inplace = True)

In [4]:
red_df = red_df[['source user@domain', 'source computer', 'destination computer']]

In [5]:
u_red_df = red_df.groupby(['source user@domain', 'source computer', 'destination computer']).size().reset_index(name='Count')

In [6]:
p_u_red_df = filter_Pikachu(u_red_df)

In [7]:
p_u_red_df

,source user@domain,source computer,destination computer,Count,src_user
0,U1025@DOM1,C17693,C1046,3,U1025
1,U1025@DOM1,C17693,C338,2,U1025
2,U1025@DOM1,C17693,C3597,1,U1025
3,U10379@C3521,C17693,C3521,2,U10379
4,U1048@DOM1,C17693,C12320,2,U1048
...,...,...,...,...,...
428,U9947@DOM1,C17693,C853,3,U9947
429,U9947@DOM1,C17693,C923,1,U9947
430,U995@DOM1,C17693,C1014,2,U995
431,U995@DOM1,C17693,C2058,1,U995


In [8]:
u_train_df = pd.read_csv('../../../lm-vol/LANL_train_unique_v2.csv')

In [9]:
p_u_train_df = filter_Pikachu(u_train_df)

In [10]:
p_u_train_df

,source user@domain,destination computer,source computer,src_user
196,U101@DOM1,C1862,C1862,U101
197,U1377@DOM1,C2162,C2162,U1377
198,U13@DOM1,C368,C368,U13
199,U1413@DOM1,C612,C265,U1413
200,U1413@DOM1,C612,C612,U1413
...,...,...,...,...
1431275,U8500@DOM1,C529,C1128,U8500
1431276,U8500@DOM1,C457,C1128,U8500
1431277,U8500@DOM1,C1114,C1115,U8500
1431278,U8500@DOM1,C1115,C1115,U8500


In [11]:
u_test_df = pd.read_csv('../../../lm-vol/LANL_test_unique_v2.csv')

In [12]:
p_u_test_df = filter_Pikachu(u_test_df)

In [13]:
p_u_test_df

,source user@domain,destination computer,source computer,src_user
109,U101@DOM1,C1862,C1862,U101
110,U10@DOM1,C229,C229,U10
111,U10@DOM1,C528,C62,U10
112,U1137@DOM1,C1065,C1065,U1137
113,U1137@DOM1,C625,C625,U1137
...,...,...,...,...
1062869,U7319@?,C528,C528,U7319
1062870,U1345@DOM1,C1640,C23653,U1345
1062871,U5462@DOM1,C1065,C17256,U5462
1062872,U5462@DOM1,C1065,C1065,U5462


In [14]:
int_df = pd.merge(p_u_red_df, p_u_test_df, how='inner', on=['source user@domain', 'destination computer', 'source computer'])

In [15]:
display(int_df)

,source user@domain,source computer,destination computer,Count,src_user_x,src_user_y
0,U1025@DOM1,C17693,C1046,3,U1025,U1025
1,U1025@DOM1,C17693,C338,2,U1025,U1025
2,U1025@DOM1,C17693,C3597,1,U1025,U1025
3,U10379@C3521,C17693,C3521,2,U10379,U10379
4,U1048@DOM1,C17693,C12320,2,U1048,U1048
...,...,...,...,...,...,...
414,U9947@DOM1,C17693,C853,3,U9947,U9947
415,U9947@DOM1,C17693,C923,1,U9947,U9947
416,U995@DOM1,C17693,C1014,2,U995,U995
417,U995@DOM1,C17693,C2058,1,U995,U995


In [16]:
train_G = nx.Graph()
graph_structure = ('source user@domain', 'source computer', 'destination computer')
for index, row in p_u_train_df.iterrows():
    client_node = row[graph_structure[0]]
    ip_node = row[graph_structure[1]]
    service_node = row[graph_structure[2]]
    train_G.add_nodes_from([ip_node, service_node, client_node])
    train_G.add_edge(ip_node, client_node)
    train_G.add_edge(ip_node, service_node)

In [17]:
graph = StellarGraph.from_networkx(train_G)

In [18]:
print(graph.info())

StellarGraph: Undirected multigraph
 Nodes: 48094, Edges: 569125

 Node types:
  default: [48094]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [569125]
        Weights: all 1 (default)
        Features: none


In [19]:
from stellargraph.data import BiasedRandomWalk
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

# init callback class
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [20]:
p = 1.0
q = 1.0
dimensions = 128
num_walks = 20
walk_length = 10
window_size = 5
num_iter = 100
workers = 32

In [21]:
rw = BiasedRandomWalk(graph)
walks = rw.run(graph.nodes(), n=num_walks, length=walk_length, p=p, q=q)
print(f"Number of random walks for Graph: {len(walks)}")

Number of random walks for Graph: 961880
Loss after epoch 0: 739948.1875
Loss after epoch 1: 569718.6875
Loss after epoch 2: 533892.0
Loss after epoch 3: 481046.625
Loss after epoch 4: 435600.75
Loss after epoch 5: 434842.5
Loss after epoch 6: 417252.25
Loss after epoch 7: 408505.75
Loss after epoch 8: 398215.75
Loss after epoch 9: 385469.5
Loss after epoch 10: 386546.0
Loss after epoch 11: 375120.5
Loss after epoch 12: 387635.5
Loss after epoch 13: 372928.0
Loss after epoch 14: 387700.0
Loss after epoch 15: 373937.0
Loss after epoch 16: 390705.0
Loss after epoch 17: 377011.5
Loss after epoch 18: 390365.5
Loss after epoch 19: 354575.0
Loss after epoch 20: 369845.0
Loss after epoch 21: 346205.0
Loss after epoch 22: 359695.0
Loss after epoch 23: 361380.0
Loss after epoch 24: 351310.0
Loss after epoch 25: 362842.0
Loss after epoch 26: 349605.0
Loss after epoch 27: 353457.0
Loss after epoch 28: 364440.0
Loss after epoch 29: 359993.0
Loss after epoch 30: 352267.0
Loss after epoch 31: 337421

In [26]:
model = Word2Vec(
    walks,
    vector_size=dimensions,
    window=2,
    min_count=0,
    sg=0,
    hs=0,
    negative=10,
    cbow_mean=1,
    workers=workers,
    epochs=num_iter,
    compute_loss=True,
    callbacks=[callback()]
)
model.get_latest_training_loss()
model.save(f"../../../lm-vol/3_10_unqiue_Pikachu_100_epochs_window_2_word2vec.model")
model.wv.save(f"../../../lm-vol/3_10_unqiue_Pikachu_100_epochs_window_2_word2vec.wordvectors")

Loss after epoch 0: 790384.125
Loss after epoch 1: 550495.375
Loss after epoch 2: 474800.375
Loss after epoch 3: 427673.125
Loss after epoch 4: 392094.75
Loss after epoch 5: 378686.75
Loss after epoch 6: 346375.0
Loss after epoch 7: 365554.75
Loss after epoch 8: 334668.75
Loss after epoch 9: 337984.0
Loss after epoch 10: 330255.0
Loss after epoch 11: 326218.0
Loss after epoch 12: 323800.0
Loss after epoch 13: 323747.0
Loss after epoch 14: 312380.0
Loss after epoch 15: 312125.5
Loss after epoch 16: 312429.0
Loss after epoch 17: 314147.5
Loss after epoch 18: 320353.5
Loss after epoch 19: 313998.5
Loss after epoch 20: 295445.5
Loss after epoch 21: 322111.0
Loss after epoch 22: 302190.5
Loss after epoch 23: 297165.0
Loss after epoch 24: 283953.0
Loss after epoch 25: 298233.0
Loss after epoch 26: 296041.0
Loss after epoch 27: 288807.0
Loss after epoch 28: 285965.0
Loss after epoch 29: 296387.0
Loss after epoch 30: 305701.0
Loss after epoch 31: 302797.0
Loss after epoch 32: 298121.0
Loss aft

In [22]:
import gensim
from gensim.models import KeyedVectors
from stellargraph.data import EdgeSplitter
import networkx as nx
from sklearn.linear_model import LogisticRegressionCV
import pandas as pd
import pickle

In [62]:
model = gensim.models.Word2Vec.load("../../../lm-vol/3_10_unqiue_Pikachu_100_epochs_window_2_word2vec.model")
wv = KeyedVectors.load('../../../lm-vol/3_10_unqiue_Pikachu_100_epochs_window_2_word2vec.wordvectors', mmap='r')

In [63]:
def operator_hadamard(u, v):
    return u * v
node_embeddings = (wv.vectors) 
# Define an edge splitter on the original graph:
edge_splitter_test = EdgeSplitter(train_G)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from graph, and obtain the
# reduced graph graph_test with the sampled links removed:
graph_test, edges_train, labels_train = edge_splitter_test.train_test_split(
    p=0.99, method="global"
)
training_edges = [operator_hadamard(wv[edge[0]], wv[edge[1]]) for edge in edges_train]

** Sampled 563433 positive and 563433 negative edges. **


In [64]:
clf = LogisticRegressionCV(cv=5, max_iter=5000, random_state=0).fit(training_edges, labels_train)
clf.score(training_edges, labels_train)

0.827406275457774

In [27]:
p_u_red_df

,source user@domain,source computer,destination computer,Count,src_user
0,U1025@DOM1,C17693,C1046,3,U1025
1,U1025@DOM1,C17693,C338,2,U1025
2,U1025@DOM1,C17693,C3597,1,U1025
3,U10379@C3521,C17693,C3521,2,U10379
4,U1048@DOM1,C17693,C12320,2,U1048
...,...,...,...,...,...
428,U9947@DOM1,C17693,C853,3,U9947
429,U9947@DOM1,C17693,C923,1,U9947
430,U995@DOM1,C17693,C1014,2,U995
431,U995@DOM1,C17693,C2058,1,U995


In [28]:
graph_structure = ('source user@domain', 'source computer', 'destination computer')
red_G = nx.Graph()
for index, row in p_u_red_df.iterrows():
    user_node = row[graph_structure[0]]
    ip_node = row[graph_structure[1]]
    service_node = row[graph_structure[2]]
    red_G.add_nodes_from([ip_node, service_node, user_node])
    red_G.add_edge(ip_node, user_node)
    red_G.add_edge(ip_node, service_node)

In [57]:
count = 0
for index, edge in enumerate(red_G.edges):
    try:
        src = wv[edge[0]]
        dest = wv[edge[1]]
        src_to_dest = operator_hadamard(src, dest)
        prob = clf.predict_proba([src_to_dest])
        if prob[0][1] < 0.1:
            count+=1
    except:
        print(None)
print(count)

None
None
None
None
None
None
None
48


In [61]:
p_u_train_df[p_u_train_df['source computer'] == 'C17693']

,source user@domain,destination computer,source computer,src_user
223071,U748@DOM1,C3007,C17693,U748
229147,U8350@DOM1,C529,C17693,U8350
230848,U8350@DOM1,C467,C17693,U8350
237566,U748@DOM1,C3644,C17693,U748
263243,U748@DOM1,C5252,C17693,U748
267347,U748@DOM1,C504,C17693,U748
280855,U748@DOM1,C1973,C17693,U748
282101,U1723@DOM1,C2742,C17693,U1723
282561,U748@DOM1,C148,C17693,U748
283410,U748@DOM1,C2651,C17693,U748
